<a href="https://colab.research.google.com/github/rifkybujana/KOPSI/blob/Research%2FModel/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jul 14 02:22:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Install Require library

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install simpletransformers

### Load and Combine Dataset

In [ ]:
import json

In [ ]:
with open('train_json.json', 'r') as f:
    fac_train = json.load(f)

with open('valid_json.json', 'r') as f:
    fac_valid = json.load(f)

In [ ]:
with open('tydiqa-train-id.json', 'r') as f:
    tydiqa_train = json.load(f)

with open('tydiqa-dev-id.json','r') as f:
    tydiqa_valid = json.load(f)

In [ ]:
train = fac_train + tydiqa_train
valid = fac_valid + tydiqa_valid

### Import BERT Model

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 64
model_args.evaluate_during_training = True
model_args.output_dir = './BERT'
model_args.best_model_dir = f'{model_args.output_dir}/best_model'
model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.do_lower_case = True

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = QuestionAnsweringModel('bert', "cahya/bert-base-indonesian-1.5G", args=model_args)

Some weights of the model checkpoint at cahya/bert-base-indonesian-1.5G were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-1.5G a

Training Model

In [ ]:
model.train_model(train, eval_data=valid)

convert squad examples to features:   0%|          | 0/8197 [00:00<?, ?it/s]Could not find answer: 'telepon ke kantor berita' vs. 'Hezb-ul Mujahedeen'
Could not find answer: 'Bau-Bau terdapat' vs. 'Pulau Makassar'
Could not find answer: 'Pantai Baturumah' vs. 'putih halus'
Could not find answer: ', Kompas - Pemerintah Indonesia dan' vs. 'Sinopec Southwest Petroleum Biro'
Could not find answer: '- Patung Kartini di Jalan' vs. 'Pemerintah DKI Jakarta'
Could not find answer: 'terdiri dari modul' vs. '200 wattjam'
Could not find answer: 'Presiden' vs. '12/9'
Could not find answer: 'Sekretaris Negara Yusril Ihza Mahendra' vs. 'Susilo Bambang Yudhoyono'
Could not find answer: 'Shenzhou yang' vs. 'Soyuz'
Could not find answer: 'Biodiesel berpotensi' vs. 'gasohol'
Could not find answer: 'Beijing , Selasa - Pemerintah' vs. 'Pemerintah China'
Could not find answer: 'menjelaskan ,' vs. '4,5 juta ton'
Could not find answer: 'Suaka Margasatwa' vs. 'Jamursbamedi'
Could not find answer: 'pukul 14.30 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/92 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 876/876 [00:05<00:00, 168.79it/s]

add example index and unique id: 100%|██████████| 876/876 [00:00<00:00, 488202.27it/s]


Running Evaluation:   0%|          | 0/111 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/111 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/92 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 876/876 [00:05<00:00, 159.62it/s]

add example index and unique id: 100%|██████████| 876/876 [00:00<00:00, 558985.29it/s]


Running Evaluation:   0%|          | 0/111 [00:00<?, ?it/s]

(460,
 {'correct': [323, 379, 389, 393, 392],
  'eval_loss': [-3.7170674800872803,
   -5.362227461359522,
   -6.769709277797389,
   -7.332642409178588,
   -8.04665638090254],
  'global_step': [92, 184, 276, 368, 460],
  'incorrect': [225, 153, 142, 134, 135],
  'similar': [328, 344, 345, 349, 349],
  'train_loss': [1.0156805515289307,
   1.6857757568359375,
   0.8048487901687622,
   0.6754388213157654,
   0.019201161339879036]})

In [ ]:
result, texts = model.eval_model(valid)

add example index and unique id: 100%|██████████| 876/876 [00:00<00:00, 686255.19it/s]


Running Evaluation:   0%|          | 0/111 [00:00<?, ?it/s]

In [ ]:
result

{'correct': 392,
 'eval_loss': -8.04665638090254,
 'incorrect': 135,
 'similar': 349}

In [ ]:
!transformers-cli login

2021-07-14 02:49:08.220963: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: Rifky
Password: 
Login successful
Your token: icJgSIALuwoYCTVLuBKkAIspTILFtgLSfvbdovOwTshpGoICkSekWISRIHvUtgGehzrqjfYpvOyNRfDPiUmCNxppQetOBDsAEblzBAcfGOsjvdzOKIrXGiGomLvuMtSb 

Your token has been saved to /root/.huggingface/token


In [ ]:
!pip install hf-lfs
!git config --global user.email "rifkybujanabisri@gmail.com"
!git config --global user.name "rifkybujana"

     |████████████████████████████████| 4.0 MB 10.3 MB/s 


In [ ]:
path = "./BERT/best_model/"

In [ ]:
from transformers import BertForQuestionAnswering, BertTokenizer

model = BertForQuestionAnswering.from_pretrained(path)
tokenizer = BertTokenizer.from_pretrained(path)

In [ ]:
tokenizer.push_to_hub("https://huggingface.co/Rifky/BERT_QA")

'https://huggingface.co/Rifky/BERT_QA/commit/6489a44d492dbbfaa5bacadde80aedd8d2f9e14a'

In [ ]:
model.push_to_hub("https://huggingface.co/Rifky/BERT_QA")

'https://huggingface.co/Rifky/BERT_QA/commit/a77bbd6d9af352de8d68c011377531e4dd26acc5'

In [27]:
!zip -r /content/BERT_LARGE.zip /content/BERT/best_model/

  adding: content/BERT/best_model/ (stored 0%)
  adding: content/BERT/best_model/config.json (deflated 46%)
  adding: content/BERT/best_model/scheduler.pt (deflated 49%)
  adding: content/BERT/best_model/vocab.txt (deflated 51%)
  adding: content/BERT/best_model/model_args.json (deflated 61%)
  adding: content/BERT/best_model/tokenizer_config.json (deflated 47%)
  adding: content/BERT/best_model/eval_results.txt (deflated 12%)
  adding: content/BERT/best_model/optimizer.pt (deflated 15%)
  adding: content/BERT/best_model/pytorch_model.bin (deflated 7%)
  adding: content/BERT/best_model/special_tokens_map.json (deflated 40%)
  adding: content/BERT/best_model/training_args.bin (deflated 48%)
